In [116]:
import pickle
import os
import time, random
import math
import requests
from bs4 import BeautifulSoup
from semanticscholar import SemanticScholar
import json

In [117]:
S2_API_KEY = os.environ['S2_API_KEY']
S2_API_KEY is not None

True

In [190]:
# search by title
sch = SemanticScholar(api_key=S2_API_KEY)
#example = 'Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills'
example = 'A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning'
print(example[:79])
results = sch.search_paper(example) # limit to 79 characters -> 넘어가면 에러남

A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Rei


In [162]:
print(results[0])
print(results[0].title)
print(results[0].year)
print(results[0].url)
print(results[0].citationCount)
print(results[0].paperId)
print(results[0].keys())

{'paperId': 'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c', 'externalIds': {'ArXiv': '2209.12072', 'DBLP': 'journals/corr/abs-2209-12072', 'DOI': '10.48550/arXiv.2209.12072', 'CorpusId': 252531963}, 'corpusId': 252531963, 'publicationVenue': {'id': '1901e811-ee72-4b20-8f7e-de08cd395a10', 'name': 'arXiv.org', 'alternate_names': ['ArXiv'], 'issn': '2331-8422', 'url': 'http://bibpurl.oclc.org/web/7130'}, 'url': 'https://www.semanticscholar.org/paper/a17a7256c04afee68f9aa0b7bfdc67fbca998b9c', 'title': 'Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills', 'abstract': '—Efﬁcient and effective exploration in continuous space is a central problem in applying reinforcement learning (RL) to autonomous driving. Skills learned from expert demon- strations or designed for speciﬁc tasks can beneﬁt the exploration, but they are usually costly-collected, unbalanced/sub-optimal, or failing to transfer to diverse tasks. However, human drivers can ada

In [122]:
# search by paperId
sch = SemanticScholar(api_key=S2_API_KEY)
example = 'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c'
results = sch.get_paper(example, fields=['title','year','paperId'])

In [123]:
results

{'paperId': 'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c', 'title': 'Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills', 'year': 2022}

In [17]:
paper = {}
paper['title'] = results[0].title
paper['paperId'] = results[0].paperId
paper['year'] = results[0].year

In [22]:
def getCitationInfo(paperId):
    headers = {'x-api-key': S2_API_KEY}
    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        headers=headers,
        params={'fields': 'citations,references,title'},
        json={"ids": [paperId]}
    )
    return r.json()[0]['citations'], r.json()[0]['references']

In [23]:
paper['citation'], paper['reference'] = getCitationInfo(paper['id'])

In [27]:
paper['citation']

[{'paperId': '32ff7e5ea4ef146cc63fdee23af1cc47e89af095',
  'title': 'NetHack is Hard to Hack'},
 {'paperId': 'e91bdc6d78cef19648c468acf9bf16a3905a5008',
  'title': 'Constraint‐based multi‐agent reinforcement learning for collaborative tasks'},
 {'paperId': '85bc55ba9ab93c09713b0891bbcf0541b8f27ea9',
  'title': 'Multi-task Hierarchical Adversarial Inverse Reinforcement Learning'},
 {'paperId': 'ac01b6062fec7a726426f9ff8c5b0dfc3f321bd7',
  'title': 'Behavior Contrastive Learning for Unsupervised Skill Discovery'},
 {'paperId': '71c508996120b06b2a056d9d073f6e6c60a671a6',
  'title': 'Progressive Transfer Learning for Dexterous In-Hand Manipulation with Multi-Fingered Anthropomorphic Hand'},
 {'paperId': 'f811132d3a737ee1c71b52706f0cd78b8904f056',
  'title': 'Learning Diverse Policies with Soft Self-Generated Guidance'},
 {'paperId': '74dd51db773ea883d9804d1845345a46ab908ccd',
  'title': 'A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning'},
 

In [25]:
paper['reference']

[{'paperId': 'bcd857d75841aa3e92cd4284a8818aba9f6c0c3f',
  'title': 'Published as a conference paper at ICLR 2018 S IMULATING A CTION D YNAMICS WITH N EURAL P ROCESS N ETWORKS'},
 {'paperId': '29e944711a354c396fad71936f536e83025b6ce0',
  'title': 'Categorical Reparameterization with Gumbel-Softmax'},
 {'paperId': '515a21e90117941150923e559729c59f5fdade1c',
  'title': 'The Concrete Distribution: A Continuous Relaxation of Discrete Random Variables'},
 {'paperId': '8fab7d7dfd233fd5d19bc2641b4c1ca74fc7bc6a',
  'title': 'Learning modular neural network policies for multi-task and multi-robot transfer'},
 {'paperId': '15b26d8cb35d7e795c8832fe08794224ee1e9f84',
  'title': 'The Option-Critic Architecture'},
 {'paperId': 'e2bd18c1039f27675bd64014117db648d969452e',
  'title': 'Learning and Transfer of Modulated Locomotor Controllers'},
 {'paperId': '136cf66392f1d6bf42da4cc070888996dc472b91',
  'title': 'On Multiplicative Integration with Recurrent Neural Networks'},
 {'paperId': '4ba25cb493ac7a

# class 화

In [242]:
class RelatedWorkAnalyzer:
    def __init__(self) -> None:
        self.paper_list = {'keypaper':[]}
        self.S2_API_KEY = os.environ['S2_API_KEY']
        self.sch = SemanticScholar(api_key=self.S2_API_KEY)
        self.is_updated = False
        self.load_paper_list()

    def build_paper_list_from_txt(self):
        if os.path.exists('./paper_list.txt'): 
            f = open('paper_list.txt', 'r')
            lines = f.readlines()
            for i, line in enumerate(lines):
                items = line.split('_')
                year = items[0]
                title = items[-1].split('\n')[0]
                self.addNewPaper(title, keypaper=True)
            self.is_updated = True
            self.update_pkl()
        else: # txt 파일이 없으면 빈 딕셔너리 반환
            print('No txt file')
            
        # Save data_dict
        self.update_pkl()

    def load_paper_list(self):
        if os.path.exists('./paper_list_from_semantic.pkl'): # pkl 파일이 존재하면 불러옴
            print('load from pkl')
            self.load_pkl()
        else: # pkl 파일이 없으면 
            print('load from txt')
            self.build_paper_list_from_txt()

    def addNewPaper(self, paper, keypaper=False):
        if ' ' in paper: # title
            title, paperId, year = self.searchByTitle(paper)
            if self.isExistInList(paperId):
                return False
        else: # paperId
            paperId = paper
            if self.isExistInList(paperId):
                return False
            title, paperId, year = self.searchByPaperId(paperId) # get the rest of info.

        # add a new paper
        citations, references = self.getCitationInfo(paperId)
        self.paper_list[paperId] = {'title': title, 
                                    'year': year, 
                                    'references': references, 
                                    'citations': citations, 
                                    'citnuminlist': 0, 
                                    'refnuminlist': 0, 
                                    'isKeypaper': keypaper}
        print('Added {} in the list'.format(title))
        if keypaper:
            self.paper_list['keypaper'].append(paperId)
            print('Added to key paper list')

        return True
    
    def load_pkl(self):
        f =  open('paper_list_from_semantic.pkl','rb')
        self.paper_list = pickle.load(f)

    def update_pkl(self):
        if self.is_updated: # 업데이트 사항이 있는 경우에만
            print('update pkl')
            with open('paper_list_from_semantic.pkl','wb') as f:
               pickle.dump(self.paper_list, f)
            self.is_updated = False
        else:
            print('no update') 

    def searchByTitle(self, title, fields=['title','year','paperId']):
        print("Searching by title... {}".format(title))
        results = self.sch.search_paper(title, fields=fields)
        paperId = results[0].paperId
        title = results[0].title
        year = results[0].year
        return title, paperId, year
    
    def searchByPaperId(self, paperId, fields=['title','year','paperId']):
        results = self.sch.get_paper(paperId, fields=fields)
        paperId = results.paperId
        title = results.title
        year = results.year
        return title, paperId, year


    def isExistInList(self, paperId):
        if paperId in self.paper_list.keys():
            print('Already exists in the list!')
            return True
        else:
            return False

    def getCitationInfo(self, paperId):
        headers = {'x-api-key': self.S2_API_KEY}
        r = requests.post(
            'https://api.semanticscholar.org/graph/v1/paper/batch',
            headers=headers,
            params={'fields': 'citations,references,title'},
            json={"ids": [paperId]}
        )
        return r.json()[0]['citations'], r.json()[0]['references']
    
    def updateCitationInfo(self, paperId, reset=False):
        keys = self.paper_list[paperId].keys()
        if 'citations' not in keys or 'references' not in keys or reset: # if there is no citation info or reset is True
            citations, references = self.getCitationInfo(paperId)
            self.paper_list[paperId]['citations'] = citations
            self.paper_list[paperId]['references'] = references
            print(self.paper_list[paperId]['title'] + "'s citations, references are updated)")
            self.is_updated = True

    def updataAllCitationInfo(self, reset=False):
        for paperId in self.paper_list.keys():
            if str(type(self.paper_list[paperId])) == "<class 'list'>":
                continue
            self.updateCitationInfo(paperId, reset)
    
    def updateRefToList(self, paperId):
        for paper in self.paper_list[paperId]['references']:
            title = paper['title']
            paperId = paper['paperId']
            if not self.isExistInList(paper['paperId']):
                self.paper_list[paperId] = {'title': title, 
                                            'year': None, 
                                            #'references': [], 
                                            #'citations': [], 
                                            'citnuminlist': 0, 
                                            'refnuminlist': 0, 
                                            'isKeypaper': False}
                print("Added {} in the list".format(title))
                self.is_updated = True


    def updateCitToList(self, paperId):
        for paper in self.paper_list[paperId]['citations']:
            title = paper['title']
            paperId = paper['paperId']
            if not self.isExistInList(paper['paperId']):
                self.paper_list[paperId] = {'title': title, 
                                            'year': None, 
                                            #'references': [], 
                                            #'citations': [], 
                                            'citnuminlist': 0, 
                                            'refnuminlist': 0, 
                                            'isKeypaper': False}
                self.is_updated = True


        

    def checkCitationInList(self, paperId): # citation 목록 중 리스트에 있는 논문이 몇 개인지 체크
        if 'citations' in self.paper_list[paperId].keys():
            self.paper_list[paperId]['citnuminlist'] = 0
            for paper in self.paper_list[paperId]['citations']:
                if paper['paperId'] in self.paper_list.keys():
                    self.paper_list[paperId]['citnuminlist'] += 1
                    self.is_updated = True

    def checkReferenceInList(self, paperId): # reference 목록 중 리스트에 있는 논문이 몇 개인지 체크
        if 'references' in self.paper_list[paperId].keys():
            self.paper_list[paperId]['refnuminlist'] = 0
            for paper in self.paper_list[paperId]['references']:
                if paper['paperId'] in self.paper_list.keys():
                    self.paper_list[paperId]['refnuminlist'] += 1
                    self.is_updated = True

    

In [250]:
rwa = RelatedWorkAnalyzer()

load from pkl


In [232]:
rwa.addNewPaper('Stochastic Neural Networks for Hierarchical Reinforcement Learning', keypaper=True)

Searching by title... Stochastic Neural Networks for Hierarchical Reinforcement Learning
Already exists in the list!


False

In [218]:
rwa.paper_list['keypaper']

['74dd51db773ea883d9804d1845345a46ab908ccd',
 '45644c7f952d2a5a5b4e594998e2e6dff9088118',
 '90c7e3f6d1b10fa31d1c2b7c3413805eee0607d8',
 'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c',
 '03eebc19358c4bddb4a987b9e94ecbcb7e58b5d2',
 'b27fd9ea29cabe6afedd01e446b96c34e956ce84',
 '0af6a63167df299a1556a560d6884ae38eda390d',
 'e6548d97d82aa2710019951eb4eac034e1747aa1',
 '1509169e79337e7e1628b266e9475aac779f9c00',
 '3b51a29424b619ec5ce29125c4b88d8e24a09328',
 '546bff6c12ea395690292f204a7e019a8b3b87a0',
 '2a88d01f3079e68ad9b5bcb1ebe56da25679e331',
 'bd2ff852e86d16df09376f2dfdc934c533bb04a2',
 '947070ff65dc9a0b0024d299acdcfa8251b5118b',
 '14444e96f58ff0ce449fd6c61abfffaec1c83f76',
 'f8cbd6e934ff43e6227bc14bc77c2934c0b66e23',
 '6a4c74430e9c9c165ef3321edf6b4e9bdc36dea9',
 '271081730bfce9117ad4432bffa8e5f18dbac133',
 '4a8b0e3b9e93c52670062b15cb2a8eae25b035a6',
 '372715a73955b7fbc1daf816bd52c0641b3ff5f2',
 '105f44c9d445de2b93d1297c2d5ac10cc776d654',
 '259b4f5ed43fda5dd3510821b40fac13021e7605',
 '45afe2d8

In [219]:
rwa.paper_list.keys()

dict_keys(['keypaper', '74dd51db773ea883d9804d1845345a46ab908ccd', '45644c7f952d2a5a5b4e594998e2e6dff9088118', '90c7e3f6d1b10fa31d1c2b7c3413805eee0607d8', 'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c', '03eebc19358c4bddb4a987b9e94ecbcb7e58b5d2', 'b27fd9ea29cabe6afedd01e446b96c34e956ce84', '0af6a63167df299a1556a560d6884ae38eda390d', 'e6548d97d82aa2710019951eb4eac034e1747aa1', '1509169e79337e7e1628b266e9475aac779f9c00', '3b51a29424b619ec5ce29125c4b88d8e24a09328', '546bff6c12ea395690292f204a7e019a8b3b87a0', '2a88d01f3079e68ad9b5bcb1ebe56da25679e331', 'bd2ff852e86d16df09376f2dfdc934c533bb04a2', '947070ff65dc9a0b0024d299acdcfa8251b5118b', '14444e96f58ff0ce449fd6c61abfffaec1c83f76', 'f8cbd6e934ff43e6227bc14bc77c2934c0b66e23', '6a4c74430e9c9c165ef3321edf6b4e9bdc36dea9', '271081730bfce9117ad4432bffa8e5f18dbac133', '4a8b0e3b9e93c52670062b15cb2a8eae25b035a6', '372715a73955b7fbc1daf816bd52c0641b3ff5f2', '105f44c9d445de2b93d1297c2d5ac10cc776d654', '259b4f5ed43fda5dd3510821b40fac13021e7605', '45afe2d8

In [220]:
for paper in rwa.paper_list['3deecaee4ec1a37de3cb10420eaabff067669e17']['references']:
    print(paper['paperId'])

bcd857d75841aa3e92cd4284a8818aba9f6c0c3f
29e944711a354c396fad71936f536e83025b6ce0
515a21e90117941150923e559729c59f5fdade1c
8fab7d7dfd233fd5d19bc2641b4c1ca74fc7bc6a
15b26d8cb35d7e795c8832fe08794224ee1e9f84
e2bd18c1039f27675bd64014117db648d969452e
136cf66392f1d6bf42da4cc070888996dc472b91
4ba25cb493ac7a03fc15d3b936257c9a6c689c1d
35da0a2001eea88486a5de677ab97868c93d0824
fddc15480d086629b960be5bff96232f967f2252
6e90fd78e8a3b98af3954aae5209703aa966603e
317cd4522b1f4a6f889743578143bb8823623f8b
1464776f20e2bccb6182f183b5ff2e15b0ae5e56
846aedd869a00c09b40f1f1f35673cb22bc87490
024006d4c2a89f7acacc6e4438d156525b60a98f
d316c82c12cf4c45f9e85211ef3d1fa62497bff8
b6b8a1b80891c96c28cc6340267b58186157e536
1389772b8a0f9c7fc43057f9da41a7d0ebf0308b
ce2e0bd9135814f4018106bc31d87902b358e251
a696aeab7b4c6bb47630663e7638fc0f60b584b8
6640f4e4beae786f301928d82a9f8eb037aa6935
bb1a17010254abfa5e1f2a17553582ce449f8e16
e0e9a94c4a6ba219e768b4e59f72c18f0a22e23d
66cdc28dc084af6507e979767755e99fe0b46b39
a2785f66c20fbdf3

In [106]:
rwa.updateRefToList('3deecaee4ec1a37de3cb10420eaabff067669e17')

Added Published as a conference paper at ICLR 2018 S IMULATING A CTION D YNAMICS WITH N EURAL P ROCESS N ETWORKS in the list
Added Categorical Reparameterization with Gumbel-Softmax in the list
Added The Concrete Distribution: A Continuous Relaxation of Discrete Random Variables in the list
Added Learning modular neural network policies for multi-task and multi-robot transfer in the list
Added The Option-Critic Architecture in the list
Added Learning and Transfer of Modulated Locomotor Controllers in the list
Added On Multiplicative Integration with Recurrent Neural Networks in the list
Added Strategic Attentive Writer for Learning Macro-Actions in the list
Added InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets in the list
Added Multimodal Compact Bilinear Pooling for Visual Question Answering and Visual Grounding in the list
Added Unifying Count-Based Exploration and Intrinsic Motivation in the list
Added VIME: Variational Informatio

In [251]:
rwa.paper_list

{'keypaper': ['74dd51db773ea883d9804d1845345a46ab908ccd',
  '45644c7f952d2a5a5b4e594998e2e6dff9088118',
  '90c7e3f6d1b10fa31d1c2b7c3413805eee0607d8',
  'a17a7256c04afee68f9aa0b7bfdc67fbca998b9c',
  '03eebc19358c4bddb4a987b9e94ecbcb7e58b5d2',
  'b27fd9ea29cabe6afedd01e446b96c34e956ce84',
  '0af6a63167df299a1556a560d6884ae38eda390d',
  'e6548d97d82aa2710019951eb4eac034e1747aa1',
  '1509169e79337e7e1628b266e9475aac779f9c00',
  '3b51a29424b619ec5ce29125c4b88d8e24a09328',
  '546bff6c12ea395690292f204a7e019a8b3b87a0',
  '2a88d01f3079e68ad9b5bcb1ebe56da25679e331',
  'bd2ff852e86d16df09376f2dfdc934c533bb04a2',
  '947070ff65dc9a0b0024d299acdcfa8251b5118b',
  '14444e96f58ff0ce449fd6c61abfffaec1c83f76',
  'f8cbd6e934ff43e6227bc14bc77c2934c0b66e23',
  '6a4c74430e9c9c165ef3321edf6b4e9bdc36dea9',
  '271081730bfce9117ad4432bffa8e5f18dbac133',
  '4a8b0e3b9e93c52670062b15cb2a8eae25b035a6',
  '372715a73955b7fbc1daf816bd52c0641b3ff5f2',
  '105f44c9d445de2b93d1297c2d5ac10cc776d654',
  '259b4f5ed43fda5dd35

In [249]:
for paperId in rwa.paper_list['keypaper']:
    rwa.checkCitationInList(paperId)
    rwa.checkReferenceInList(paperId)
    print(rwa.paper_list[paperId]['title'])
    print(rwa.paper_list[paperId]['citnuminlist'])
    print(rwa.paper_list[paperId]['refnuminlist'])
rwa.update_pkl()

A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning
0
8
Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning
0
3
Hierarchical Adversarial Inverse Reinforcement Learning
0
2
Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills
0
8
Advanced Skills through Multiple Adversarial Motion Priors in Reinforcement Learning
0
1
ASPiRe: Adaptive Skill Priors for Reinforcement Learning
0
10
Cascaded Compositional Residual Learning for Complex Interactive Behaviors
0
4
CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
0
7
MPR-RL: Multi-Prior Regularized Reinforcement Learning for Knowledge Transfer
0
0
Planning to Practice: Efficient Online Fine-Tuning by Composing Goals in Latent Space
0
1
Priors, Hierarchy, and Information Asymmetry for Skill Transfer in Reinforcement Learning
0
10
Relay Hindsight Experience Replay: Self-Guided Continual